# Projet Fixmatch

In [1]:
# !pip install torchview torchsummary torchvision kornia torchmetrics matplotlib tqdm path graphviz opencv-python scikit-learn optuna

In [2]:
# system
import sys
sys.path.append('..')

# data
import numpy as np

# deep learning
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

# torchvision
import torchvision

# plotting
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm

# utils and model
from utils.utils import (
    seedEverything,
    compute_mean_std,
    data_transform,
    normalize,
    plot_images,
)

from utils.model import ConvNN

# FIXMATCH
from fixmatch_DA import fixmatch_DA_train

# os
import os
import path

# sklearn
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.cluster import SpectralClustering
from sklearn.metrics.pairwise import pairwise_distances

%load_ext autoreload
%autoreload 2

In [3]:
# Set device
if ((int(torch.__version__.split(".")[0]) >= 2) or (int(torch.__version__.split(".")[1]) >= 13)) and torch.has_mps:
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print(device)

mps


In [4]:
IMG_SHAPE = (3, 32, 32)
# See Table 4
TAU = 0.9
LAMBDA_U = 3
MU = 4
BATCH_SIZE = 64
LR = 0.03
BETA = 0.9
WEIGHT_DECAY = 0.0005

In [5]:
# Download both datasets
trainset = torchvision.datasets.CIFAR10(
    root='../data', train=True, download=True, transform=data_transform())
testset = torchvision.datasets.CIFAR10(
    root='../data', train=False, download=True, transform=data_transform())

# convert to data loaders
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

Files already downloaded and verified
Files already downloaded and verified


In [6]:
# compute mean and std
if not os.path.exists('../data/mean.pt'):
    mean, std = compute_mean_std(trainloader)
    torch.save(mean, '../data/mean.pt')
    torch.save(std, '../data/std.pt')
else:
    mean, std = torch.load('../data/mean.pt'), torch.load('../data/std.pt')

# to numpy
mean, std = mean.numpy(), std.numpy()

print(f"mean: {mean}, std: {std}")

mean: [0.49133518 0.48210305 0.44649622], std: [0.24633394 0.24276228 0.26066196]


In [7]:
# define classes
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


In [8]:
torch_models = './results/models/' 
if not os.path.exists(torch_models):
    os.makedirs(torch_models)

In [9]:
# load full model
model_full = ConvNN(return_features=True).to(device)
model_full.load_state_dict(torch.load(f'../SUP/results/models/model_100.pth', map_location=device))

<All keys matched successfully>

## IV. Semi-Supervised Learning: Fixmatch with Spectral Clustering on feature space

### IV.0 Spectral Clustering

In [10]:
def perform_spectral_clustering(
        model_full: ConvNN,
        trainloader: torch.utils.data.DataLoader,
        budget: float = 0.01
        ) -> np.ndarray:

    # Compute the output embeddings from the model
    print("Extracting embeddings from the model")
    embeddings = []
    pbar = tqdm(trainloader, total=len(trainloader))
    for i, (train_data) in enumerate(pbar):
        images, _ = train_data
        images = normalize(mean, std)(images)
        images = images.to(device)

        # Forward pass
        outputs = model_full(images).detach().cpu().numpy()
        embeddings.append(outputs)

    # Stack the embeddings
    embeddings_array = np.vstack(embeddings)

    # Compute pairwise distance matrix
    pairwise_distances_matrix = pairwise_distances(embeddings_array)

    # Perform spectral clustering
    spectral_clustering = SpectralClustering(
        n_clusters=10, affinity='precomputed', assign_labels='discretize', random_state=0
    )
    spectral_clustering.fit(pairwise_distances_matrix)

    # Obtain the cluster labels
    cluster_labels = spectral_clustering.labels_

    # Compute the cluster centers
    print("Computing cluster centers")
    cluster_centers = []
    for cluster_i in range(10):
        cluster_i_indices = np.where(cluster_labels == cluster_i)[0]
        cluster_i_embeddings = embeddings_array[cluster_i_indices]
        cluster_i_center = np.mean(cluster_i_embeddings, axis=0)
        cluster_centers.append(cluster_i_center)

    cluster_centers = np.vstack(cluster_centers)

    # Compute the distance between each data point and cluster centers
    distances = pairwise_distances(embeddings_array, cluster_centers)

    # Filter distance to closest cluster center according to prediction
    distances_filtered = np.array(
        [distances[i, cluster_labels[i]] for i in range(len(cluster_labels))]
    )

    # Show histogram and horizontal lines indicating X% of the data
    plt.hist(distances_filtered, bins=100)
    plt.axvline(np.percentile(distances_filtered, 90), color='red', label='90%')
    plt.axvline(np.percentile(distances_filtered, 95), color='blue', label='95%')
    plt.axvline(np.percentile(distances_filtered, 99), color='green', label='99%')
    plt.xlabel('Distance to assigned cluster center')
    plt.ylabel('Number of data points')
    plt.title('Histogram of distances to assigned cluster center')
    plt.legend()
    plt.show()

    # Select the top budget fraction of samples closest to their assigned cluster centers
    # indices = np.argsort(distances_filtered)[:int(budget * len(distances_filtered))]

    # select half of the samples close to their assigned cluster centers and half of the samples far from their assigned cluster centers
    indices = np.argsort(distances_filtered)[:int(budget * len(distances_filtered) / 2)] + np.argsort(distances_filtered)[int(budget * len(distances_filtered) / 2):int(budget * len(distances_filtered))]

    # save clusters
    torch.save(cluster_labels, f'./results/clustering/clusters.pth')
    torch.save(indices, f'./results/clustering/indices_{int(budget*100)}.pth')


    return indices

### IV.1 Fixmatch on 10% train data with KMeans Clustering 

In [11]:
# fix the seed
seedEverything()

SUBSET_PROP = 0.10

# load indices
indices_10 = perform_spectral_clustering(model_full, trainloader, budget=0.10)

Extracting embeddings from the model


  0%|          | 0/782 [00:00<?, ?it/s]

: 

: 

In [26]:


trainset_sup = torch.utils.data.Subset(trainset, indices_10)

trainset_unsup, _ = torch.utils.data.random_split(trainset, [1, 0])

labeled_dataloader = DataLoader(
    trainset_sup,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=0
)

unlabeled_dataloader = DataLoader(
    trainset_unsup,
    batch_size=MU*BATCH_SIZE,
    shuffle=True,
    num_workers=0
)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataset.py:348: UserWarning: Length of split at index 1 is 0. This might result in an empty dataset.
  warnings.warn(f"Length of split at index {i} is 0. "


In [ ]:
EPOCHS = 300

model = ConvNN().to(device)

# criterion and optimizer
labeled_criterion = nn.CrossEntropyLoss(reduction='none')
unlabeled_criterion = nn.CrossEntropyLoss(reduction='none')

optimizer = torch.optim.SGD(model.parameters(), lr=LR, momentum=BETA, weight_decay=WEIGHT_DECAY, nesterov=True)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

In [ ]:
fixmatch_DA_train(
    model=model,
    labeled_dataloader=labeled_dataloader,
    unlabeled_dataloader=unlabeled_dataloader,
    testloader=testloader,
    labeled_criterion=labeled_criterion,
    unlabeled_criterion=unlabeled_criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    epochs=EPOCHS,
    mean=mean,
    std=std,
    lambda_u=LAMBDA_U,
    tau=TAU,
    mu=MU,
    batch_size=BATCH_SIZE,
    name='10_KMEANS'
)

In [ ]:
# open results
model_10 = torch.load('./results/models/model_DA_10_KMEANS.pth')
train_losses_10 = torch.load('./results/metrics/train_losses_DA_10_KMEANS.pth')
train_accuracies_10 = torch.load('./results/metrics/train_accuracies_DA_10_KMEANS.pth')
test_losses_10 = torch.load('./results/metrics/test_losses_DA_10_KMEANS.pth')
test_accuracies_10 = torch.load('./results/metrics/test_accuracies_DA_10_KMEANS.pth')

In [ ]:
# plot losses and accuracies
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

ax1.plot(train_losses_10, label="train")
ax1.plot(test_losses_10, label="test")
ax1.set_title("Loss")

ax2.plot(train_accuracies_10, label="train")
ax2.plot(test_accuracies_10, label="test")
ax2.set_title("Accuracy")

plt.legend()
plt.show()

# save plot
fig.savefig(f"/results/figures/losses_accuracies_DA_10_KMEANS.png")

In [ ]:
# plot confusion matrix
model.eval()  # Set the model to evaluation mode
test_correct = 0
test_total = 0
y_true = []
y_pred = []

with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        # normalize
        images = normalize(mean, std)(images)
        
        outputs = model_10(images)
        _, predicted = outputs.max(1)
        test_total += labels.size(0)
        test_correct += predicted.eq(labels).sum().item()

        y_true.append(labels.cpu().numpy())
        y_pred.append(predicted.cpu().numpy())
    
    y_true = np.concatenate(y_true)
    y_pred = np.concatenate(y_pred)

    fig, ax = plt.subplots(figsize=(10, 10))
    cm = confusion_matrix(y_true, y_pred, normalize='true')
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes)
    disp.plot(ax=ax)
    plt.tight_layout()
    plt.show()

    # save plot
    fig.savefig(f"figures/confusion_matrix_DA_10_KMEANS.png")

In [ ]:
# Evaluation on the test set
test_accuracy = 100.0 * test_correct / test_total
print(f'Test Accuracy: {test_accuracy}%')

test_image, test_labels = testloader.__iter__().__next__()
test_image = test_image.to(device)
test_image = normalize(mean, std)(test_image)
outputs_test = model_10(test_image)
label_pred_test = outputs_test.argmax(dim=1)

fig1 = plot_images(test_image, test_labels, label_pred_test, classes, figure_name=f"Test score with Fixmatch - {int(SUBSET_PROP*100)}% - {test_accuracy:.2f}%")
fig1.savefig(f"./figures/test_score_DA_10_KMEANS.png")

### III.2 Fixmatch on 5% train data

In [ ]:
# fix the seed
seedEverything()

SUBSET_PROP = 0.05

# load indices
indices_05 = perform_KMeans_selection(model_full, trainloader, budget=0.05)

trainset_sup = torch.utils.data.Subset(trainset, indices_05)

trainset_unsup, _ = torch.utils.data.random_split(trainset, [1, 0])

labeled_dataloader = DataLoader(
    trainset_sup,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=0
)

unlabeled_dataloader = DataLoader(
    trainset_unsup,
    batch_size=MU*BATCH_SIZE,
    shuffle=True,
    num_workers=0
)

In [ ]:
EPOCHS = 300

model = ConvNN().to(device)

# criterion and optimizer
labeled_criterion = nn.CrossEntropyLoss(reduction='none')
unlabeled_criterion = nn.CrossEntropyLoss(reduction='none')

optimizer = torch.optim.SGD(model.parameters(), lr=LR, momentum=BETA, weight_decay=WEIGHT_DECAY, nesterov=True)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

In [ ]:
fixmatch_DA_train(
    model=model,
    labeled_dataloader=labeled_dataloader,
    unlabeled_dataloader=unlabeled_dataloader,
    testloader=testloader,
    labeled_criterion=labeled_criterion,
    unlabeled_criterion=unlabeled_criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    epochs=EPOCHS,
    mean=mean,
    std=std,
    lambda_u=LAMBDA_U,
    tau=TAU,
    mu=MU,
    batch_size=BATCH_SIZE,
    name='05_KMEANS'
)

In [ ]:
# open results
model_05 = torch.load('./results/models/model_DA_05_KMEANS.pth')
train_losses_05 = torch.load('./results/metrics/train_losses_DA_05_KMEANS.pth')
train_accuracies_05 = torch.load('./results/metrics/train_accuracies_DA_05_KMEANS.pth')
test_losses_05 = torch.load('./results/metrics/test_losses_DA_05_KMEANS.pth')
test_accuracies_05 = torch.load('./results/metrics/test_accuracies_DA_05_KMEANS.pth')

In [ ]:
# plot losses and accuracies
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

ax1.plot(train_losses_05, label="train")
ax1.plot(test_losses_05, label="test")
ax1.set_title("Loss")

ax2.plot(train_accuracies_05, label="train")
ax2.plot(test_accuracies_05, label="test")
ax2.set_title("Accuracy")

plt.legend()
plt.show()

# save plot
fig.savefig(f"/results/figures/losses_accuracies_DA_05_KMEANS.png")

In [ ]:
# plot confusion matrix
model.eval()  # Set the model to evaluation mode
test_correct = 0
test_total = 0
y_true = []
y_pred = []

with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        # normalize
        images = normalize(mean, std)(images)
        
        outputs = model_05(images)
        _, predicted = outputs.max(1)
        test_total += labels.size(0)
        test_correct += predicted.eq(labels).sum().item()

        y_true.append(labels.cpu().numpy())
        y_pred.append(predicted.cpu().numpy())
    
    y_true = np.concatenate(y_true)
    y_pred = np.concatenate(y_pred)

    fig, ax = plt.subplots(figsize=(10, 10))
    cm = confusion_matrix(y_true, y_pred, normalize='true')
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes)
    disp.plot(ax=ax)
    plt.tight_layout()
    plt.show()

    # save plot
    fig.savefig(f"figures/confusion_matrix_DA_05_KMEANS.png")

In [ ]:
# Evaluation on the test set
test_accuracy = 100.0 * test_correct / test_total
print(f'Test Accuracy: {test_accuracy}%')

test_image, test_labels = testloader.__iter__().__next__()
test_image = test_image.to(device)
test_image = normalize(mean, std)(test_image)
outputs_test = model_05(test_image)
label_pred_test = outputs_test.argmax(dim=1)

fig1 = plot_images(test_image, test_labels, label_pred_test, classes, figure_name=f"Test score with Fixmatch - {int(SUBSET_PROP*100)}% - {test_accuracy:.2f}%")
fig1.savefig(f"./figures/test_score_DA_05_KMEANS.png")

### III.3 Fixmatch on 1% train data

In [ ]:
# fix the seed
seedEverything()

SUBSET_PROP = 0.01

# load indices
indices_01 = perform_KMeans_selection(model_full, trainloader, budget=0.01)

trainset_sup = torch.utils.data.Subset(trainset, indices_01)

trainset_unsup, _ = torch.utils.data.random_split(trainset, [1, 0])

labeled_dataloader = DataLoader(
    trainset_sup,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=0
)

unlabeled_dataloader = DataLoader(
    trainset_unsup,
    batch_size=MU*BATCH_SIZE,
    shuffle=True,
    num_workers=0
)

In [ ]:
EPOCHS = 300

model = ConvNN().to(device)

# criterion and optimizer
labeled_criterion = nn.CrossEntropyLoss(reduction='none')
unlabeled_criterion = nn.CrossEntropyLoss(reduction='none')

optimizer = torch.optim.SGD(model.parameters(), lr=LR, momentum=BETA, weight_decay=WEIGHT_DECAY, nesterov=True)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

In [ ]:
fixmatch_DA_train(
    model=model,
    labeled_dataloader=labeled_dataloader,
    unlabeled_dataloader=unlabeled_dataloader,
    testloader=testloader,
    labeled_criterion=labeled_criterion,
    unlabeled_criterion=unlabeled_criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    epochs=EPOCHS,
    mean=mean,
    std=std,
    lambda_u=LAMBDA_U,
    tau=TAU,
    mu=MU,
    batch_size=BATCH_SIZE,
    name='01_KMEANS'
)

In [ ]:
# open results
model_01 = torch.load('./results/models/model_DA_01_KMEANS.pth')
train_losses_01 = torch.load('./results/metrics/train_losses_DA_01_KMEANS.pth')
train_accuracies_01 = torch.load('./results/metrics/train_accuracies_DA_01_KMEANS.pth')
test_losses_01 = torch.load('./results/metrics/test_losses_DA_01_KMEANS.pth')
test_accuracies_01 = torch.load('./results/metrics/test_accuracies_DA_01_KMEANS.pth')

In [ ]:
# plot losses and accuracies
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

ax1.plot(train_losses_01, label="train")
ax1.plot(test_losses_01, label="test")
ax1.set_title("Loss")

ax2.plot(train_accuracies_01, label="train")
ax2.plot(test_accuracies_01, label="test")
ax2.set_title("Accuracy")

plt.legend()
plt.show()

# save plot
fig.savefig(f"/results/figures/losses_accuracies_DA_01_KMEANS.png")

In [ ]:
# plot confusion matrix
model.eval()  # Set the model to evaluation mode
test_correct = 0
test_total = 0
y_true = []
y_pred = []

with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        # normalize
        images = normalize(mean, std)(images)
        
        outputs = model_01(images)
        _, predicted = outputs.max(1)
        test_total += labels.size(0)
        test_correct += predicted.eq(labels).sum().item()

        y_true.append(labels.cpu().numpy())
        y_pred.append(predicted.cpu().numpy())
    
    y_true = np.concatenate(y_true)
    y_pred = np.concatenate(y_pred)

    fig, ax = plt.subplots(figsize=(10, 10))
    cm = confusion_matrix(y_true, y_pred, normalize='true')
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes)
    disp.plot(ax=ax)
    plt.tight_layout()
    plt.show()

    # save plot
    fig.savefig(f"figures/confusion_matrix_DA_01_KMEANS.png")

In [ ]:
# Evaluation on the test set
test_accuracy = 100.0 * test_correct / test_total
print(f'Test Accuracy: {test_accuracy}%')

test_image, test_labels = testloader.__iter__().__next__()
test_image = test_image.to(device)
test_image = normalize(mean, std)(test_image)
outputs_test = model_01(test_image)
label_pred_test = outputs_test.argmax(dim=1)

fig1 = plot_images(test_image, test_labels, label_pred_test, classes, figure_name=f"Test score with Fixmatch - {int(SUBSET_PROP*100)}% - {test_accuracy:.2f}%")
fig1.savefig(f"./figures/test_score_DA_01_KMEANS.png")